In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
from config import username
from config import password

In [2]:
# Read in USA covid csv data
csv_file = "Resources/us_covid19_daily.csv"
covid_us_df = pd.read_csv(csv_file)
covid_us_df.head()

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,totalTestResults,lastModified,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,20201206,56,14534035,161986294,13592.0,101487.0,585676.0,20145.0,31946.0,7094.0,...,204063869,2020-12-06T24:00:00Z,0,0,1138,2256,1172590,176771,1634532,9cf16504f91958e803a2197daf8c2528a4eddc18
1,20201205,56,14357264,160813704,13433.0,101190.0,583420.0,19950.0,31831.0,7005.0,...,202429337,2020-12-05T24:00:00Z,0,0,2445,3316,1526995,211073,2169756,6249216c5f097c94ce33a811dab011a483a42404
2,20201204,56,14146191,159286709,12714.0,101276.0,580104.0,19858.0,31608.0,6999.0,...,200259581,2020-12-04T24:00:00Z,0,0,2563,4652,1260657,224831,1854869,ae30ea088584335ba4d57ee927f8dbda6add74db
3,20201203,56,13921360,158026052,15106.0,100755.0,575452.0,19723.0,31276.0,6867.0,...,198404712,2020-12-03T24:00:00Z,0,0,2706,5331,1238465,210204,1828230,0f253d185ecb336cdd18a4c61996eda1b7eef13b
4,20201202,56,13711156,156787587,14368.0,100322.0,570121.0,19680.0,31038.0,6855.0,...,196576482,2020-12-02T24:00:00Z,0,0,2733,5028,982032,195796,1459202,477c17b6302d0485195e77ecf8270a974f7a3c82


In [3]:
# Read in India covid json data
json_file = "Resources/covid19india-QueryResult.json"
india_covid_df = pd.read_json(json_file)
india_covid_df

,dailyconfirmed,dailydeceased,dailyrecovered,date,totalconfirmed,totaldeceased,totalrecovered
0,1,0,0,30 January,1,0,0
1,0,0,0,31 January,1,0,0
2,0,0,0,01 February,1,0,0
3,1,0,0,02 February,2,0,0
4,1,0,0,03 February,3,0,0
...,...,...,...,...,...,...,...
410,24437,130,20186,15 March,11409517,158301,11025464
411,28869,187,17746,16 March,11438386,158488,11043210
412,35838,171,17793,17 March,11474224,158659,11061003
413,39687,156,20356,18 March,11513911,158815,11081359


In [4]:
# clean data by filtering for columns in common with india dataset and dropping rows with nulls
clean_covid_us_df = covid_us_df[["date", "positive", "recovered", "death"]].copy()
clean_covid_us_df = clean_covid_us_df.dropna()
clean_covid_us_df = clean_covid_us_df.sort_values(by=['positive']).reset_index()

In [5]:
clean_covid_us_df = clean_covid_us_df.drop(['index'], axis=1)
pd.set_option('display.max_rows', clean_covid_us_df.shape[0]+1)

In [6]:
# add 'ID' column to serve as primary key in our postgres database

clean_covid_us_df.insert(0, "id", clean_covid_us_df.index+1)
clean_covid_us_df

,id,date,positive,recovered,death
0,1,20200325,75242,146.0,1058.0
1,2,20200326,92976,97.0,1374.0
2,3,20200327,112220,2418.0,1782.0
3,4,20200328,131826,3145.0,2333.0
4,5,20200329,151270,4076.0,2837.0
5,6,20200330,172740,4560.0,3425.0
6,7,20200331,197924,5666.0,4332.0
7,8,20200401,224040,7084.0,5336.0
8,9,20200402,252085,8586.0,6514.0
9,10,20200403,283923,10861.0,7799.0


In [7]:
# clean data by filtering for columns in common with india dataset and dropping rows with nulls
clean_india_covid_df = india_covid_df[["date", "totalconfirmed", "totalrecovered", "totaldeceased"]].copy()
filter_1 = clean_india_covid_df['totalconfirmed']<9703858
clean_india_covid_df = clean_india_covid_df[filter_1]

In [8]:
# Filtered out for dates before march 26, 2020 and after December 6, 2020.
filter_2 = clean_india_covid_df['totalconfirmed']>571
clean_india_covid_df = clean_india_covid_df[filter_2].reset_index()

In [9]:
clean_india_covid_df = clean_india_covid_df.drop(['index'], axis=1)
pd.set_option('display.max_rows', clean_india_covid_df.shape[0]+1)

In [10]:
# Added ID column to serve as primary key
clean_india_covid_df.insert(0, "id", clean_india_covid_df.index+1)
clean_india_covid_df

,id,date,totalconfirmed,totalrecovered,totaldeceased
0,1,25 March,657,43,11
1,2,26 March,730,50,16
2,3,27 March,883,75,19
3,4,28 March,1019,85,24
4,5,29 March,1139,102,27
5,6,30 March,1326,137,41
6,7,31 March,1635,150,47
7,8,01 April,2059,169,53
8,9,02 April,2545,191,69
9,10,03 April,3105,230,83


In [12]:
# connect to local database
rds_connection_string = f"{username}:{password}@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
# Check for tables
engine.table_names()

['covid_usa', 'covid_india']

In [14]:
# Load usa csv and india json into the postgres database
clean_covid_us_df.to_sql(name = 'covid_usa', con = engine, if_exists = 'append', index = False)
clean_india_covid_df.to_sql(name = 'covid_india', con = engine, if_exists = 'append', index = False)

In [15]:
# Query the database to ensure data loaded properly
pd.read_sql_query('select * from covid_india', con=engine).head()

,id,date,totalconfirmed,totalrecovered,totaldeceased
0,1,25 March,657,43,11
1,2,26 March,730,50,16
2,3,27 March,883,75,19
3,4,28 March,1019,85,24
4,5,29 March,1139,102,27


In [16]:
# Query the database to ensure data loaded properly
pd.read_sql_query('select * from covid_usa', con=engine).head()

,id,date,positive,recovered,death
0,1,20200325,75242,146,1058
1,2,20200326,92976,97,1374
2,3,20200327,112220,2418,1782
3,4,20200328,131826,3145,2333
4,5,20200329,151270,4076,2837
